<a href="https://colab.research.google.com/github/rajdeepd/sk-samples/blob/main/colab/05-using-the-planner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to the Planner

The Planner is one of the fundamental concepts of the Semantic Kernel.

It makes use of the collection of native and semantic functions that have been registered to the kernel and using AI, will formulate a plan to execute the given ask.

From our own testing, planner works best with more powerful models like `gpt4` but sometimes you might get working plans with cheaper models like `gpt-35-turbo`. We encourage you to implement your own versions of the planner and use different models that fit your user needs.  

Read more about planner [here](https://aka.ms/sk/concepts/planner)

In [2]:
!python -m pip install semantic-kernel==0.3.12.dev0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 16.2 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.5.2
    Uninstalling pydantic-2.5.2:
      Successfully uninstalled pydantic-2.5.2
  Attempting uninstall: openai
    Found existing installation: openai 1.3.8
    Uninstalling openai-1.3.8:
      Successfully uninstalled openai-1.3.8
  Attempting uninstall: semantic-kernel
    Found existing installation: semantic-kernel 0.4.1.dev0
    Uninstalling semantic-kernel-0.4.1.dev0:
      Successfully uninstalled semantic-kernel-0.4.1.dev0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 req

In [4]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, AzureChatCompletion

kernel = sk.Kernel()

useAzureOpenAI = False
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

# Configure AI backend used by the kernel
if useAzureOpenAI:

    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_chat_service("gpt-3.5", AzureChatCompletion(deployment, endpoint, api_key))
else:
    api_key, org_id = OPENAI_API_KEY,""
    kernel.add_chat_service("gpt-3.5", OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id))

## It all begins with an ask

In [5]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas. She speaks French so write it in French.
Convert the text to uppercase"""

### Providing skills to the planner
The planner needs to know what skills are available to it. Here we'll give it access to the `SummarizeSkill` and `WriterSkill` we have defined on disk. This will include many semantic functions, of which the planner will intelligently choose a subset.

You can also include native functions as well. Here we'll add the TextSkill.

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# note: using skills from the samples folder
skills_directory = "/content/drive/MyDrive/skills"

In [9]:
from semantic_kernel.core_skills.text_skill import TextSkill

#skills_directory = "../../samples/skills/"
summarize_skill = kernel.import_semantic_skill_from_directory(skills_directory, "SummarizeSkill")
writer_skill = kernel.import_semantic_skill_from_directory(skills_directory, "WriterSkill")
text_skill = kernel.import_skill(TextSkill(), "TextSkill")

Define your ASK. What do you want the Kernel to do?

# Basic Planner

 Let's start by taking a look at a basic planner. The `BasicPlanner` produces a JSON-based plan that aims to solve the provided ask sequentially and evaluated in order.

In [10]:
from semantic_kernel.planning.basic_planner import BasicPlanner
planner = BasicPlanner()

In [11]:
basic_plan = await planner.create_plan_async(ask, kernel)

In [12]:
print(basic_plan.generated_plan)

{
    "input": "Valentine's Day Date Ideas",
    "subtasks": [
        {"function": "WriterSkill.Brainstorm"},
        {"function": "WriterSkill.EmailTo", "args": {"recipient": "significant_other"}},
        {"function": "WriterSkill.Translate", "args": {"language": "French"}},
        {"function": "TextSkill.uppercase"}
    ]
}


You can see that the Planner took my ask and converted it into an JSON-based plan detailing how the AI would go about solving this task, making use of the skills that the Kernel has available to it.

As you can see in the above plan, the AI has determined which functions to call in order to fulfill the user ask. The output of each step of the plan becomes the input to the next function.

Let's also define an inline skill and have it be available to the Planner. Be sure to give it a function name and skill name.

In [13]:
sk_prompt = """
{{$input}}

Rewrite the above in the style of Shakespeare.
"""
shakespeareFunction = kernel.create_semantic_function(sk_prompt, "shakespeare", "ShakespeareSkill",
                                                      max_tokens=2000, temperature=0.8)

Let's update our ask using this new skill

In [14]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas.
She likes Shakespeare so write using his style. She speaks French so write it in French.
Convert the text to uppercase."""

new_plan = await planner.create_plan_async(ask, kernel)

In [15]:
print(new_plan.generated_plan)

{
    "input": "Valentine's Day Date Ideas",
    "subtasks": [
        {"function": "WriterSkill.Brainstorm"},
        {"function": "ShakespeareSkill.shakespeare"},
        {"function": "WriterSkill.Translate", "args": {"language": "French"}},
        {"function": "TextSkill.uppercase"}
    ]
}


### Executing the plan

Now that we have a plan, let's try to execute it! The Planner has a function called `execute_plan`.

In [16]:
results = await planner.execute_plan_async(new_plan, kernel)

In [17]:
print(results)

1. ASSUREZ-VOUS DE PARTICIPER À UN DÎNER ROMANTIQUE AUX CHANDELLES DANS UN RESTAURANT CHIC,
2. UN PIQUE-NIQUE DANS LE PARC, AVEC DES SANDWICHS EN FORME DE CŒUR ET DES CHOCOLATS,
3. PROFITEZ D'UNE JOURNÉE DE DÉTENTE EN COUPLE, AVEC DES MASSAGES ET DES SOINS DU VISAGE,
4. GRIMPEZ LORS D'UNE RANDONNÉE PITTORESQUE, ET UNE FOIS AU SOMMET DE LA MONTAGNE, PROFITEZ D'UN PIQUE-NIQUE,
5. ASSISTEZ À UN COURS DE CUISINE ENSEMBLE, ET APPRENEZ À PRÉPARER UN REPAS SPÉCIAL POUR LA SAINT-VALENTIN,
6. LAISSEZ-VOUS ALLER À UN MARATHON DE FILMS ROMANTIQUES À LA MAISON,
7. PARTEZ POUR UNE ESCAPADE D'UN WEEK-END DANS UN CHALET CONFORTABLE EN PLEINE NATURE,
8. ORGANISEZ UNE SOIRÉE SURPRISE AVEC UNE CHASSE AU TRÉSOR À THÈME À TRAVERS LA VILLE,
9. RÉDIGEZ DES LETTRES D'AMOUR ET ÉCHANGEZ-LES LORS D'UN REPAS ROMANTIQUE À LA MAISON,
10. LANCEZ-VOUS DANS UNE VISITE DE DÉGUSTATION DE VINS ET PROFITEZ D'UNE JOURNÉE TRANQUILLE À EXPLORER LES VIGNOBLES.


# The Plan Object Model

To build more advanced planners, we need to introduce a proper Plan object that can contain all the necessary state and information needed for high quality plans.

To see what that object model is, look at (https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planning/plan.py)

# Sequential Planner

The sequential planner is an XML-based step-by-step planner. You can see the prompt used for it here (https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planning/sequential_planner/Skills/SequentialPlanning/skprompt.txt)

In [23]:
#from semantic_kernel.planning import SequentialPlanner
#planner = SequentialPlanner(kernel)

In [22]:
#sequential_plan = await planner.create_plan_async(goal=ask)

PlanningException: ignored

To see the steps that the Sequential Planner will take, we can iterate over them and print their descriptions

In [21]:
#for step in sequential_plan._steps:
#    print(step.description, ":", step._state.__dict__)

NameError: ignored

Let's ask the sequential planner to execute the plan.

In [ ]:
#result = await sequential_plan.invoke_async()

In [ ]:
#print(result)

# Action Planner

The action planner takes in a list of functions and the goal, and outputs a **single** function to use that is appropriate to meet that goal.

In [ ]:
from semantic_kernel.planning import ActionPlanner
planner = ActionPlanner(kernel)

Let's add more skills to the kernel

In [ ]:
from semantic_kernel.core_skills import FileIOSkill, MathSkill, TextSkill, TimeSkill
kernel.import_skill(MathSkill(), "math")
kernel.import_skill(FileIOSkill(), "fileIO")
kernel.import_skill(TimeSkill(), "time")
kernel.import_skill(TextSkill(), "text")

In [ ]:
ask = "What is the sum of 110 and 990?"

In [ ]:
plan = await planner.create_plan_async(goal=ask)

In [ ]:
result = await plan.invoke_async()

In [ ]:
print(result)

# Stepwise Planner

Stepwise Planner is based off the paper from MRKL (Modular Reasoning, Knowledge and Language) and is similar to other papers like ReACT (Reasoning and Acting in Language Models). At the core, the stepwise planner allows for the AI to form "thoughts" and "observations" and execute actions based off those to achieve a user's goal. This continues until all required functions are complete and a final output is generated.

See a video walkthrough of Stepwise Planner [here.](https://youtu.be/DG_Ge1v0c4Q?si=T1CHaAm1vV0mWRHu)

In [ ]:
from semantic_kernel.planning import StepwisePlanner
from semantic_kernel.planning.stepwise_planner.stepwise_planner_config import (
    StepwisePlannerConfig,
)

Let's create a Bing Search native skill that we can pass in to the Kernel.

Make sure you have a Bing Search API key in your `.env` file

(https://www.microsoft.com/en-us/bing/apis/bing-web-search-api)

In [ ]:
class WebSearchEngineSkill:
    """
    A search engine skill.
    """
    from semantic_kernel.orchestration.sk_context import SKContext
    from semantic_kernel.skill_definition import sk_function, sk_function_context_parameter

    def __init__(self, connector) -> None:
        self._connector = connector

    @sk_function(
        description="Performs a web search for a given query", name="searchAsync"
    )
    @sk_function_context_parameter(
        name="query",
        description="The search query",
    )
    async def search_async(self, query: str, context: SKContext) -> str:
        query = query or context.variables.get("query")[1]
        result = await self._connector.search_async(query, num_results=5, offset=0)
        return str(result)

In [ ]:
from semantic_kernel.connectors.search_engine import BingConnector

BING_API_KEY = sk.bing_search_settings_from_dot_env()
connector = BingConnector(BING_API_KEY)
kernel.import_skill(WebSearchEngineSkill(connector), skill_name="WebSearch")

Let's also add a couple more skills

In [ ]:
from semantic_kernel.core_skills.math_skill import MathSkill
from semantic_kernel.core_skills.time_skill import TimeSkill

kernel.import_skill(TimeSkill(), "time")
kernel.import_skill(MathSkill(), "math")

In [ ]:
planner = StepwisePlanner(
    kernel, StepwisePlannerConfig(max_iterations=10, min_iteration_time_ms=1000)
)

Now let's do a more complicated ask that will require planner to make a call to Bing to get the latest information.

In [ ]:
ask = """How many total championships combined do the top 5 teams in the NBA have?"""

plan = planner.create_plan(goal=ask)

In [ ]:
result = await plan.invoke_async()

In [ ]:
print(result)

Let's see the steps that the AI took to get to the answer.

In [ ]:
for index, step in enumerate(plan._steps):
    print("Step:", index)
    print("Description:",step.description)
    print("Function:", step.skill_name + "." + step._function.name)
    if len(step._outputs) > 0:
        print( "  Output:\n", str.replace(result[step._outputs[0]],"\n", "\n  "))